In [270]:
import geopandas as gpd, folium, requests, pandas as pd, json, numpy as np

import gzip
import os
import shutil
from geopandas import GeoDataFrame
from shapely.geometry import Point


# If needed


## Data Retrieval

In [271]:
counties = 'BOUNDARIES_counties.geojson' #Source: https://gis-data-portal-austin.hub.arcgis.com/datasets/boundaries-counties/explore

In [272]:
districts = gpd.read_file('/Users/valhalla/data_science/ayrw/aisd_district/20_21_high_updt_v2.shp')

In [273]:
recharge_zone = pd.read_csv('/Users/valhalla/data_science/ayrw/ed_aq_recharge_zone.csv')

In [274]:
council_districts = gpd.read_file('/Users/valhalla/data_science/ayrw/council_districts/BOUNDARIES_single_member_districts.shp')

In [275]:
df_markers = pd.read_csv('test_water_data_clean.csv')

In [276]:
#districts

In [277]:
#council_districts

In [278]:
#recharge_zone.head()

In [279]:
#districts.geometry.centroid

In [280]:
#df_markers.head(1)

## Base Map Creation 

In [281]:
# it is hard to determine what level the zoom should be, 12, focuses on the points and shows most of the school districts. 11 shows all of the school districts, but is less focused on the area. The 
# thickness of the district lines hosuld be updated when this i

m=folium.Map(location=(30.263, -97.75196987), zoom_start= 12.0, tiles=None)#, tiles='stamenterrain')

In [282]:
folium.raster_layers.TileLayer(tiles='stamenterrain', name='Austin Water Qualtiy Data Map', opacity=.45, control=False).add_to(m)

## Title

In [283]:
title_html = '''
             <h3 align="left" style="font-size:30px"><b>Program Impact Map</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))

#m

## Water Quality Site Data

In [284]:
df_wqs = df_markers[df_markers.type == 'WQS'].reset_index()
#df_wqs

In [285]:
#creating a function for the tooltip for the wqs's

#LCRA data link can be replaced with other datain the future

def tooltip_wqs(row):

    i = row
    site = df_wqs.name.iloc[i]
    sample_dt = df_wqs.date.iloc[i]
    sample_air_temp = df_wqs.air_temp.iloc[i]
    sample_water_temp = df_wqs.water_temp.iloc[i]
    sample_flow = df_wqs.flow.iloc[i]
    sample_ph = df_wqs.ph.iloc[i]
    sample_nitrates = df_wqs.nitrates.iloc[i]
    sample_do_avg = df_wqs.do_avg.iloc[i]
    sample_do_percentage_sat = df_wqs.do_percentage_sat.iloc[i]
    sample_cond = df_wqs.cond.iloc[i]
    sample_last_rain = df_wqs.last_rain.iloc[i]
    sample_field_rain_gauge_in = df_wqs.field_rain_gauge_in.iloc[i]
    sample_secchi = df_wqs.secchi.iloc[i]
    sample_trans = df_wqs.transparency.iloc[i]
    sample_e_coli = df_wqs.e_coli.iloc[i]
    
    lcra_data = df_wqs.lcra_data.iloc[i]

    left_col_color = "#f2f0d3" #change these
    right_col_color = "#39AADD"

    #source: https://towardsdatascience.com/folium-map-how-to-create-a-table-style-pop-up-with-html-code-76903706b88a

    html = """<!DOCTYPE html>
    <html>
        <head>
            <h3 style="font-size:20px; margin-bottom:10"; width="200px">{}</h4>""".format(site) + """
        </head>
        <table style="font-size:18px; width: 400px;"> <!--,'font-family:"Courier New", Courier, monospace; font-size:200%'>-->
        
            <tbody>
                <tr>
                    <td style="background-color: """+ left_col_color +""";">Date of Last Sample</span></td>
                    <td style="width: 150px;background-color: """+ right_col_color +""";"><span style="color: #ffffff;">{}</td>""".format(sample_dt) + """
                </tr>
                <tr>
                    <td style="background-color: """+ left_col_color +""";">Air Temp, C&deg</span></td>
                    <td style="width: 150px;background-color: """+ right_col_color +""";"><span style="color: #ffffff;">{}</td>""".format(sample_air_temp) + """
                </tr>
                <tr>
                    <td style="background-color: """+ left_col_color +""";">Water temp, C&deg </span></td>
                    <td style="width: 150px;background-color: """+ right_col_color +""";"><span style="color: #ffffff;">{}</td>""".format(sample_water_temp) + """
                </tr>
                <tr>
                    <td style="background-color: """+ left_col_color +""";">Flow Severity</span></td>
                    <td style="width: 150px;background-color: """+ right_col_color +""";"><span style="color: #ffffff;">{}</td>""".format(sample_flow) + """
                </tr>
                <tr>
                    <td style="background-color: """+ left_col_color +""";">pH, SU</span></td>
                    <td style="width: 150px;background-color: """+ right_col_color +""";"><span style="color: #ffffff;">{}</td>""".format(sample_ph) + """
                </tr>
                <tr>
                    <td style="background-color: """+ left_col_color +""";">Nitrates, mg/L</span></td>
                    <td style="width: 150px;background-color: """+ right_col_color +""";"><span style="color: #ffffff;">{}</td>""".format(sample_nitrates) + """
                </tr>
                <tr>
                    <td style="background-color: """+ left_col_color +""";">Dissolved Oxygen, mg/L</span></td>
                    <td style="width: 150px;background-color: """+ right_col_color +""";"><span style="color: #ffffff;">{}</td>""".format(sample_do_avg) + """
                </tr>
                <tr>
                    <td style="background-color: """+ left_col_color +""";">Dissolved Oxygen % Saturation</span></td>
                    <td style="width: 150px;background-color: """+ right_col_color +""";"><span style="color: #ffffff;">{}</td>""".format(sample_do_percentage_sat) + """
                </tr>
                <tr>
                    <td style="background-color: """+ left_col_color +""";">Conductivity, &microS/cm </span></td>
                    <td style="width: 150px;background-color: """+ right_col_color +""";"><span style="color: #ffffff;">{}</td>""".format(sample_cond) + """
                </tr>
                <tr>
                    <td style="background-color: """+ left_col_color +""";">Last Rain, days</span></td>
                    <td style="width: 150px;background-color: """+ right_col_color +""";"><span style="color: #ffffff;">{}</td>""".format(sample_last_rain) + """
                </tr>
                <tr>
                    <td style="background-color: """+ left_col_color +""";">Field Rain Gauge, in</span></td>
                    <td style="width: 150px;background-color: """+ right_col_color +""";"><span style="color: #ffffff;">{}</td>""".format(sample_field_rain_gauge_in) + """
                </tr>
                <tr>
                    <td style="background-color: """+ left_col_color +""";">Secchi</span></td>
                    <td style="width: 150px;background-color: """+ right_col_color +""";"><span style="color: #ffffff;">{}</td>""".format(sample_secchi) + """
                </tr>
                <tr>
                    <td style="background-color: """+ left_col_color +""";">Transparency</span></td>
                    <td style="width: 150px;background-color: """+ right_col_color +""";"><span style="color: #ffffff;">{}</td>""".format(sample_trans) + """
                </tr>
                <tr>
                    <td style="background-color: """+ left_col_color +""";">E Coli</span></td>
                    <td style="width: 150px;background-color: """+ right_col_color +""";"><span style="color: #ffffff;">{}</td>""".format(sample_e_coli) + """
                </tr>
            </tbody>
        </table>
        <body>
            <br>
            <font size="4">
            <a href="""+ lcra_data +""" target="_blank"><b>LCRA Data Link</b></a> 
        </body>
    </html>
    """
    return html
    
    

In [286]:
#df_wqs

In [287]:
# icons: https://getbootstrap.com/docs/3.3/components/

feature_group_wqs = folium.FeatureGroup('Water Quality Sites')
for i in range(len(df_wqs)):
    #print(i)
    
    html = tooltip_wqs(i)
    tooltips = folium.features.Tooltip(text=df_wqs.name[i], sticky=True, style="font-size:18px")
    popup = folium.Popup(folium.Html(html, script=True), max_width=500)
    folium.Marker(
                location=[df_wqs.lat[i], df_wqs.long[i]],popup=popup, tooltip=tooltips,
                icon=folium.Icon(icon="tint", color='blue')).add_to(feature_group_wqs)
    
    #print(feature_group_wqs)
    
    feature_group_wqs.add_to(m)

## Restoration Site Data

In [288]:
df_rs = df_markers[df_markers.type == 'RS'].reset_index()
#df_rs

In [289]:
#creating a function for the tooltip for the wqs's

def tooltip_rs(row):

    i = row
    site = df_rs.name.iloc[i]
    info = df_rs.information.iloc[i]

    html = """<!DOCTYPE html>
    <html>
        <head>
            <h3 width="200px">{}</h3>""".format(site) + """
        </head>

        <body>
            <br>
            <font size="4">
            <p style="margin-bottom:10"; width="200px">{}</h4>""".format(info) + """
        </body>
    </html>
    """
    return html
    
    

In [290]:
feature_group_rs = folium.FeatureGroup('Restoration Sites')

for i in range(len(df_rs)):

    html = tooltip_rs(i)
    tooltips = folium.features.Tooltip(text=df_rs.name[i], sticky=True, style="font-size:18px")
    popup = folium.Popup(folium.Html(html, script=True), max_width=500)
    folium.Marker(
                location=[df_rs.lat[i], df_rs.long[i]],popup=popup, tooltip=tooltips,
                icon=folium.Icon(icon="tree-deciduous", color='green')).add_to(feature_group_rs),
    #print(feature_group_rs)
    feature_group_rs.add_to(m)

## Site of Interest Data

In [291]:
df_soi = df_markers[df_markers.type == 'SOI'].reset_index()

In [292]:
#creating a function for the tooltip for the soi's

def tooltip_soi(row):

    i = row
    site = df_soi.name.iloc[i]
    info = df_soi.information.iloc[i]

    html = """<!DOCTYPE html>
    <html>
        <head>
            <h3 width="200px">{}</h3>""".format(site) + """
        </head>

        <body>
            <br>
            <font size="4">
            <p style="margin-bottom:10"; width="200px">{}</h4>""".format(info) + """
        </body>
    </html>
    """
    return html

In [293]:
feature_group_soi = folium.FeatureGroup('Sites of Interest')

for i in range(len(df_soi)):

    html = tooltip_soi(i)
    tooltips = folium.features.Tooltip(text=df_soi.name[i], sticky=True, style="font-size:18px")
    popup = folium.Popup(folium.Html(html, script=True), max_width=500)
    folium.Marker(
                location=[df_soi.lat[i], df_soi.long[i]],popup=popup, tooltip=tooltips,
                icon=folium.Icon(icon="globe", color='purple')).add_to(feature_group_soi)
    #print(feature_group_rs)
    feature_group_soi.add_to(m)

## HS Attendance District Data 

In [294]:
districts = districts.rename(columns={'HIGH_NAME':'ATTENDANCE_DISTRICT'})

In [295]:
folium.GeoJson(districts[['geometry','ATTENDANCE_DISTRICT']],
               name='Austin High School Attendance Districts',
              style_function=lambda feature: {
        'fillColor': '#F0712A',#'#F1B123',
        'color': '#F0712A',
        'weight': 3},
        tooltip=folium.features.GeoJsonTooltip(fields = ['ATTENDANCE_DISTRICT'], style="font-size:16px",aliases= ['Attendance District:'])
    ).add_to(m)

## Council District Data

In [296]:
folium.GeoJson(council_districts[['geometry','COUNCIL_DI']],
               name='City Council Districts',
               show = False,
              style_function=lambda feature: {
        'fillColor': '#F1B123',
        'color': '#F1B123',
        'weight': 3},
        tooltip=folium.features.GeoJsonTooltip(fields = ['COUNCIL_DI'])
    ).add_to(m)

## Counties Data

In [297]:
#Unsure that I want to use this

#folium.GeoJson(counties, show=False, name='Counties').add_to(m)


## Title

## Layer Control

In [298]:
#SHould this be collapsed? especially if there is a legend



folium.LayerControl(collapsed=True).add_to(m)


## Legend

In [299]:
from branca.element import Template, MacroElement

#from: https://nbviewer.org/gist/talbertc-usgs/18f8901fc98f109f2b71156cf3ac81cd

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#39AADD;opacity:0.7;'></span>Water Quality Sites</li>
    <li><span style='background:#72AF25;opacity:0.7;'></span>Restoration Sites</li>
    <li><span style='background:#D251B8;opacity:0.7;'></span>Sites of Interest</li>
    <li><span style='background:#F0712A;opacity:0.7;'></span>Austin High School Attendance Districts</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 18px;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 18px;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 18px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

m.get_root().add_child(macro)

In [300]:
m.save('ayrw.html')

In [301]:

# m=folium.Map(location=(30.3, -97.72096987), zoom_start= 11, tile =None)#, tiles='stamenterrain')

In [302]:
m

### Unused Code

In [303]:
# # shapefiles_path = '/Users/valhalla/Downloads/isd_bd_2020_v1'
# # output_path = '/Users/valhalla'

# #Would like to remember where I got this function from

# def shp_2_json(shapefiles_path: str, output_path: str, compress=True) -> None:
#     """
#     Function transforms given directory with shapefiles into geojsons.
#     :param shapefiles_path: (str) Path to the directory with shapefiles.
#     :param output_path: (str) Path to the output geojson file.
#     :param compress: (bool) Compress output JSON into gz archive.
#     """
    
#     shp = '.shp'
#     geojson = 'json'
#     shapefiles = [fname for fname in os.listdir(shapefiles_path) if fname.endswith(shp)]
    
#     for sfile in shapefiles:
#         # Get and create paths
#         spath = os.path.join(shapefiles_path, sfile)
#         gjfile = sfile[:-3]+geojson
#         gjpath = os.path.join(output_path, gjfile)
        
#         # Load & save
#         gdf = gpd.read_file(spath)
#         gdf.to_file(gjpath, driver='GeoJSON')
#         if compress:
#             with open(gjpath, 'rb') as f_in:
#                 gzipped = gjpath + '.gz'
#                 with gzip.open(gzipped, 'wb') as f_out:
#                     shutil.copyfileobj(f_in, f_out)
#             # remove JSON
#             os.remove(gjpath)

# shp_2_json(shapefiles_path, output_path, compress=False)

# schools = '20_21_high_updt_v2.geojson'

In [304]:
council_districts = shp_2_json('/Users/valhalla/data_science/ayrw/council_districts/', '/Users/valhalla', compress=False)

NameError: name 'shp_2_json' is not defined

In [ ]:
council_districts
